In [3]:
!pip install langchain

In [58]:
from google.colab import userdata
geminiKey = userdata.get('GOOGLE_API_KEY')

In [59]:
import os
os.environ["GOOGLE_API_KEY"] = geminiKey

In [8]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [9]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [13]:
import requests

In [25]:
respose = requests.get("https://raw.githubusercontent.com/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/main/RAG%20App%20using%20Langchain%20OpenAI%20FAISS/state_of_the_union.txt")

In [26]:
respose

<Response [200]>

In [27]:
filename = "state_of_the_union.txt"  # save file in current working directory
with open(filename, "w", encoding="utf-8") as f:
  f.write(respose.text)

In [28]:
with open("state_of_the_union.txt","r", encoding="utf8") as f:
  data = f.read()

In [29]:
loader = TextLoader('state_of_the_union.txt')

In [30]:
document = loader.load()

In [31]:
document

[Document(metadata={'source': 'state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determin

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
text_splitter = RecursiveCharacterTextSplitter( chunk_size=500,chunk_overlap=50)

In [34]:
text_chunks = text_splitter.split_documents(document)

In [36]:
print(text_chunks[0].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.


In [37]:
!pip install langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [54]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
import os

In [63]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [56]:
! pip install FAISS-cpu

In [64]:
vector_store = FAISS.from_documents(text_chunks,embedding=embeddings)

In [66]:
retriever=vector_store.as_retriever()

In [67]:
from langchain.prompts import ChatPromptTemplate

In [68]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [69]:
prompt=ChatPromptTemplate.from_template(template)

In [70]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [71]:
output_parser=StrOutputParser()

In [72]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [74]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [75]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

'The United States is providing military, economic, and humanitarian assistance to Ukraine in its fight for freedom. The U.S. has committed over $1 billion in direct assistance to Ukraine. The U.S. is also working with allies to enforce powerful economic sanctions against Russia. The U.S. is a member of NATO and prepared extensively and carefully with allies.'

In [76]:
rag_chain.invoke("Who is kp oli?")

'I\'m sorry, but the provided documents do not contain any information about a person named "kp oli." Therefore, I cannot answer your question.'